In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import warnings 
import sys
 
warnings.filterwarnings('ignore')

In [71]:
# 数据路径
df_target_path = '/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/price target.dta'

In [ ]:
# target price数据较小，直接读取处理
df_tar = pd.read_stata(df_target_path, columns = ['cusip', 'anndats', 'value', 'estimid', 'alysnam'])

# 加入‘date’数据方便计算
df_tar['date'] = df_tar['anndats']
df_tar = df_tar.drop('anndats', axis = 1)

# flag用来记录修改过的数据
df_tar['flag'] = False

In [74]:
# 形成trade_list

df_prc_2011 = pd.read_stata(df_price_path_3, columns = ['date'], iterator = True)
df_prc_1 = df_prc_2011.get_chunk(100000)

df_prc_2001 = pd.read_stata(df_price_path_2, columns = ['date'], iterator = True)
df_prc_2 = df_prc_2001.get_chunk(100000)

df_prc_1975 = pd.read_stata(df_price_path_1, columns = ['date'], iterator = True)
df_prc_3 = df_prc_1975.get_chunk(100000)

In [75]:
print(df_prc_1['date'].max())
print(df_prc_1['date'].min())

print(df_prc_2['date'].max())
print(df_prc_2['date'].min())

print(df_prc_3['date'].max())
print(df_prc_3['date'].min())

2020-02-28 00:00:00
2011-01-03 00:00:00
2010-12-31 00:00:00
2001-01-02 00:00:00
2000-12-29 00:00:00
1999-02-19 00:00:00


In [76]:
# 文件中的时间跨度：
# 这个时间跨度是事先通过观察数据确定的--只属于xxxx--20xx.dta数据集
# 手动得到

max_date_2011 = pd.Timestamp(2020, 2, 28)
min_date_2011 = pd.Timestamp(2011, 1, 3)

max_data_2001 = pd.Timestamp(2010, 12, 31)
min_data_2001 = pd.Timestamp(2001, 1, 2)

max_data_1975 = pd.Timestamp(2000, 12, 29)
min_data_1975 = pd.Timestamp(2001, 1, 2)

In [77]:
# 将stock price的所有trading date生成一个list
# 思路是通过这个list来调整target price data
trading_date_1 = pd.DataFrame(df_prc_1.groupby('date').nunique()['date'])
trading_date_1 = trading_date_1.drop('date', axis = 1).reset_index()
trade_list_1 = list(trading_date_1['date'].values)

trading_date_2 = pd.DataFrame(df_prc_2.groupby('date').nunique()['date'])
trading_date_2 = trading_date_2.drop('date', axis = 1).reset_index()
trade_list_2 = list(trading_date_2['date'].values)

trading_date_3 = pd.DataFrame(df_prc_3.groupby('date').nunique()['date'])
trading_date_3 = trading_date_3.drop('date', axis = 1).reset_index()
trade_list_3 = list(trading_date_3['date'].values)

In [78]:
# 生成trading_date_list
# trade_list包含所有股票数据交易日
trade_list = trade_list_1 + trade_list_2 + trade_list_3

In [79]:
# sort for comparing
trade_list.sort()

In [350]:
#函数：寻找下一个交易日
def find_nearest(list, date):
    date = np.datetime64(date)
    
    # 防止越界
    # 此时是分析师数据的日期大于目前的股票数据
    if date > trade_list[-1] or date < trade_list[0]:
        return np.nan
        
    while (date in list) == False:
        date = np.datetime64(date + pd.Timedelta(days=1))
        # print the wrong date to inform
        #print(date)

    return np.datetime64(date)

# 主函数：调整dataframe时间列
def process_time(trading_list, df): 
    count = 0
    for index in df.index:
        time = np.datetime64(df['date'][index])

        if (time in trading_list) == False and index < len(df) - 1:
            # find the last valid date
            new_time = find_nearest(trading_list, time)
            df['flag'][index] = True
            print('Wps {} '.format(new_time))
            df['date'][index] = new_time
            count = count + 1

        if (time in trading_list) == False and index == len(df) - 1:
            df['date'][index] = np.nan
    # 计算调整过多少个
    print('there are {} deviation in target price data'.format(count))
    df.to_csv('/Users/wangjunjie/Desktop/python lab/data_modify_price/data_target_price.csv')

In [352]:
# waste time a lot :-<
# we then get the modified target price data for later.
# we save the data
target_price_modi = process_time(trade_list, df_tar)

In [3]:
# modify之后的target price数据
# 另外导入了一次，开始下面的merge及分析

target_price_modi = (pd.read_stata('/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/updated_target.dta'))

In [4]:
df_price_path_1 = '/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/stock_data/new/1999-2000_new.dta'
df_price_path_2 = '/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/stock_data/new/2001-2010_new.dta'
df_price_path_3 = '/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/stock_data/new/2011-2020_new.dta'

df_1 = pd.read_stata(df_price_path_1, columns =  ['permno', 'ncusip', 'date', 'prc', 'ret'])
df_2 = pd.read_stata(df_price_path_2, columns =  ['permno', 'ncusip', 'date', 'prc', 'ret'])
df_3 = pd.read_stata(df_price_path_3, columns =  ['permno', 'ncusip', 'date', 'prc', 'ret'])

In [38]:
frames = [df_1, df_2, df_3]
df_price = pd.concat(frames)

#df_output = pd.merge(df_price, target_price_modi, on = ['ncusip', 'date'])

In [39]:
# 股票信息存储在df_price_simple中
df_price_simple = df_price[['ncusip','ret','date','permno','prc']]

In [40]:
df_price_simple = df_price_simple.dropna(axis=0) 
#df_price_simple = df_price_simple.groupby(['ncusip','date'])['ret'].mean()

In [41]:
df_price_simple = df_price_simple.reset_index()

In [42]:
df_price_simple['index'] = df_price_simple.index
df_price_simple

,index,ncusip,ret,date,permno,prc
0,0,29274A10,0.014085,1999-02-19,10001.0,9.000000
1,1,29274A10,-0.013889,1999-02-22,10001.0,-8.875000
2,2,29274A10,-0.014085,1999-02-23,10001.0,8.750000
3,3,29274A10,0.000000,1999-02-24,10001.0,8.750000
4,4,29274A10,0.014286,1999-02-25,10001.0,-8.875000
...,...,...,...,...,...,...
37596615,37596615,88160R10,-0.074595,2020-02-24,93436.0,833.789978
37596616,37596616,88160R10,-0.040634,2020-02-25,93436.0,799.909973
37596617,37596617,88160R10,-0.026390,2020-02-26,93436.0,778.799988
37596618,37596618,88160R10,-0.128146,2020-02-27,93436.0,679.000000


In [43]:
df_output = pd.merge(target_price_modi, df_price_simple, on = ['ncusip', 'date'])

In [44]:
df_output_1 = df_output[df_output['horizon'] == '12']

In [52]:
df_output_1['prc'] = abs(df_output_1['prc'])

df_output_1['last_day_price'] = df_output_1['prc'] / (1 + df_output_1['ret'])
df_output_1['expect_12_stock_return'] = df_output_1['value'] / df_output_1['last_day_price'] - 1
#df_output_1 = df_output_1.drop('level_0', axis = 1)

# 我们获得了第一个dataset！
#df_output_1.to_stata('output_standard.dta')

In [53]:
# 接着merge index数据
df_index_path = '/Users/wangjunjie/Desktop/private/比赛 & 调研 & 助研/RA/张英广老师/暑假/delayed alpha/data/stock market indexes-daily.dta/stock market indexes-daily.dta'
df_index = pd.read_stata(df_index_path, columns = ['ewretd','date'])
df_index['index_mar'] = df_index.index

df_output_index = pd.merge(df_output_1, df_index, on = ['date'])

In [60]:
# 我们对value取平均值，其他值实际上在给定date、ncusip后都相同，因此不影响
df_output_real = df_output_index.groupby(['ncusip','date']).mean()
df_output_real = df_output_real.reset_index()

# 保存一下
#df_output_real.to_stata('new_expect_12_data.dta')

In [61]:
index = df_output_real['index_y']
df_output_real = df_output_real.drop(['index_x','index_y'], axis = 1)
df_output_real['index'] = index

In [62]:
df_output_real

,ncusip,date,level_0,value,ret,permno,prc,last_day_price,expect_12_stock_return,ewretd,index_mar,index
0,00030710,2014-12-15,11736.0,35.0,-0.008678,14945.0,29.700001,29.959999,0.168224,-0.011453,23523.0,24766701.0
1,00030710,2015-01-08,11737.0,37.0,0.025489,14945.0,29.370001,28.639999,0.291899,0.013099,23539.0,24766717.0
2,00030710,2015-02-25,11738.0,41.0,0.122369,14945.0,37.330002,33.259998,0.232712,0.003326,23571.0,24766749.0
3,00030710,2015-03-05,11739.0,41.0,-0.017146,14945.0,32.099998,32.660000,0.255358,0.002430,23577.0,24766755.0
4,00030710,2015-05-21,11740.0,45.0,0.063323,14945.0,37.950001,35.689999,0.260857,0.000886,23631.0,24766809.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1209024,Y9530810,2019-12-17,5459112.0,27.0,-0.091721,15857.0,15.250000,16.790001,0.608100,0.002179,24783.0,25504721.0
1209025,Y9530810,2019-12-30,5459113.5,20.0,-0.494624,15857.0,7.990000,15.810001,0.265022,-0.001236,24791.0,25504729.0
1209026,Y9530810,2019-12-31,5459115.5,15.0,0.003129,15857.0,8.015000,7.990000,0.877347,0.005045,24792.0,25504730.0
1209027,Y9530810,2020-01-02,5459117.0,20.0,-0.004367,15857.0,7.980000,8.015000,1.495321,0.005470,24793.0,25504731.0


In [63]:
df_use = df_output_real[['date','ret','ewretd','expect_12_stock_return', 'ncusip', 'index', 'index_mar','permno']]

# 删除没有expect-12-return的数据
df_use = df_use[~np.isnan(df_use['expect_12_stock_return'])]

# 按照exp-12-return分组之前按照其排序
df_group = df_use.sort_values(by = 'expect_12_stock_return').reset_index()

In [64]:
df_use = df_use[~np.isnan(df_use['ret'])]

In [680]:
df_use.groupby('ncusip').count()

,date,ret,ewretd,expect_12_stock_return,index,index_mar,permno
ncusip,,,,,,,
00030710,36,36,36,36,36,36,36
00036020,60,60,60,60,60,60,60
00036110,202,202,202,202,202,202,202
00037520,196,196,196,196,196,196,196
00040010,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...
Y8897Y80,3,3,3,3,3,3,3
Y8977Y10,19,19,19,19,19,19,19
Y9369110,70,70,70,70,70,70,70


In [65]:
def split(origin_list, n):
    lista = []
    
    if len(origin_list) % n == 0:
        cnt = len(origin_list) // n
    else:
        cnt = len(origin_list) // n + 1
 
    for i in range(0, n):
        lista.append(origin_list[i*cnt:(i+1)*cnt])
    return lista

In [66]:
list_group = split(df_group, 8)
list_group

[        level_0       date       ret    ewretd  expect_12_stock_return  \
 0          9125 2004-08-11 -0.157556 -0.006461               -1.000000   
 1       1142769 2011-05-06 -0.029630  0.004575               -1.000000   
 2        486625 2003-06-16 -0.120603  0.011857               -1.000000   
 3        361953 2011-02-15 -0.023605 -0.003015               -1.000000   
 4       1142768 2011-05-02 -0.237374 -0.006003               -1.000000   
 ...         ...        ...       ...       ...                     ...   
 151124   673898 2003-12-15 -0.012496 -0.008949               -0.000357   
 151125   179766 2006-04-28  0.012139  0.003117               -0.000357   
 151126   198183 2011-02-18  0.010175  0.001088               -0.000357   
 151127   988583 2014-08-14 -0.002499  0.002516               -0.000357   
 151128   359048 2007-02-15  0.000178  0.001444               -0.000357   
 
           ncusip       index  index_mar   permno  
 0       00252W10  12903018.0    20918.0  8391

In [87]:
df_stock = df_price_simple

In [651]:
# 通过此函数在事实上算出来股票的cummulative return
def RollingPrior(ret):
    log_ret = np.log(ret + 1)
    log_cumret = log_ret.shift(270).rolling(window = 281).sum() 
    return np.exp(log_cumret)

df_stock['cumret'] = df_stock.groupby('ncusip')['ret'].transform(RollingPrior)
df_index['cumret'] = df_index['ewretd'].transform(RollingPrior)

In [599]:
# 实际上 280 + 271 = 551之后才会有数据
# 因此后面需要调整
# 数据的损失是因为算法问题导致的，此处因为抛弃的数据较少，暂时先用这种方法
# 如果珍惜数据的话，需要另写函数
df_index.head(552)

,ewretd,date,index_mar,cumret
0,NaN,1925-12-31,0,NaN
1,0.009516,1926-01-02,1,NaN
2,0.005780,1926-01-04,2,NaN
3,-0.001927,1926-01-05,3,NaN
4,0.001182,1926-01-06,4,NaN
...,...,...,...,...
547,-0.001406,1927-10-27,547,NaN
548,-0.015248,1927-10-28,548,NaN
549,-0.005279,1927-10-29,549,NaN
550,0.007050,1927-10-31,550,NaN


In [565]:
# 我们可以最终获得股票的cummulative return
# 通过index, index_mar建立两个数据集的索引关系

for df in list_group:
    df['cumret'] = np.nan
    df['mar_cumret'] = np.nan
    df['cum'] = np.nan
    
    count = 0
    
    for index in df.index:
        # 使用index_refer作为桥梁传递
        
        if df['index'][index] < 270 or df['index_mar'][index] < 270:
            df['cum'] = np.nan
            count += 1
            continue
        
        else:
            
            df['cumret'][index] = df_stock['cumret'][df['index'][index] - 270]
            df['mar_cumret'][index] = df_index['cumret'][df['index_mar'][index] - 270]
        
            # 生成每一个股票，每一天的cum数据
            df['cum'] = df['cumret'][index] - df['mar_cumret'][index]
        #print(df['cum'][index])
    print('there is', count, 'NaN in list')

there is 28 NaN in list
there is 18 NaN in list
there is 23 NaN in list
there is 30 NaN in list
there is 21 NaN in list
there is 34 NaN in list
there is 39 NaN in list
there is 75 NaN in list


In [81]:
# 有一个小问题就是目前无法排除最后270天的数据，实际上这些数据是没法算的
# 但是因为前面rolling的算法，使得最后的cumret在最后的270天也存在
time_max = trade_list[-271]
print(time_max)
for i in range(0,8):
    
    # 处理时间问题
    list_group[i] = list_group[i][list_group[i]['date'] <= time_max]

2019-02-01T00:00:00.000000000


In [82]:
for i in range(0,8):
    list_group[i].drop('level_0', axis = 1, inplace = True)

In [83]:
df_group.drop('level_0', axis = 1, inplace = True)

In [95]:
df_group_1 = df_group[["permno", "date", "expect_12_stock_return"]]
df_group_1

,permno,date,expect_12_stock_return
0,83918.0,2004-08-11,-1.000000
1,59483.0,2011-05-06,-1.000000
2,87147.0,2003-06-16,-1.000000
3,86260.0,2011-02-15,-1.000000
4,59483.0,2011-05-02,-1.000000
...,...,...,...
1209024,81049.0,2016-03-30,890.443577
1209025,16034.0,2020-02-19,913.923390
1209026,16034.0,2018-11-07,975.061763
1209027,17094.0,2018-02-12,1002.805783


In [1]:
# compute the cumulative market-adjusted return
def cumAdjReturn(series):
    print(series['index'])
    ix = df_stock['permno'] == series['permno']
    df = df_stock[ix].reset_index()
    
    # check if the timespan is wide enough
    if (len(df) < 281 or
           df['date'].iloc[10] > series['date'] or
           df['date'].iloc[-271] < series['date']):
        return np.nan
    
    # find the index of each target price date
    ix_target = df[df['date'] == series['date']].index.tolist()[0]
    ix_market = df_index[df_index.date == series['date']].index.tolist()[0]
    
    # we get the price in range(-10,270) in df
    df1 = df.iloc[(ix_target-10):(ix_target+271)]
    df2 = df_index.iloc[(ix_market-10):(ix_market+271)]
    
    # compute cumulative returns into an array
    # cumsum is fitable tool for this task
    df_cum_ret = np.exp(np.log(df1['ret'] + 1).cumsum()).reset_index(drop = True)
    df_mkt_ret = np.exp(np.log(df2['ewretd'] + 1).cumsum()).reset_index(drop = True)
    return  df_cum_ret - df_mkt_ret

# randomly pick a chunk of data to generate plot
# chunk size is in discretion
def randomPlot(size):
    df_sample = df_group_1.sample(size).reset_index(drop = True)
    df_cumret = df_sample.reset_index().apply(cumAdjReturn, axis = 1)
    df_sample = pd.concat([df_sample, df_cumret], axis = 1)
    
    df_sample.sort_values(by = ['expect_12_stock_return'], inplace = True)
    df_sample.dropna(axis = 0, how = "any", inplace = True)
    df_sample = df_sample.reset_index(drop = True).reset_index()
    df_sample['group'] = df_sample['index'] // (len(df_sample) / 8)
    #df_avgret = df_sample.groupby('group')['cumret'].apply(np.mean)
    df_avgret = df_sample.iloc[:,4:].groupby('group').agg(np.nanmean)
    
    df_plot = pd.DataFrame(columns = range(0,281), index = range(1,9))
    for i in range(0,8): df_plot.iloc[i] = df_avgret.iloc[i]
    df_plot.columns = range(-10,271)
    print(df_plot)
    
    plot = df_plot.T.plot(figsize = (20,16))
    plot.get_figure().savefig("rand_" + str(size) + ".png")
    print("rand_" + str(size) + ".png successfully saved.")

In [ ]:
randomPlot(1000)